神经网络构造

Module 类是 torch.nn 模块里提供的一个模型构造类，是所有神经网络模块的基类，可以继承它来定义我们想要的模型

In [ ]:
"""
定义 MLP 类重载 Module 类的 __init__ 函数和 forward 函数。
分别用于创建模型参数和定义前向计算（正向传播）。
MLP 类定义了一个具有两个隐藏层的多层感知机
"""
import torch
import torch.nn as nn

class MLP(nn.Module):
    
    def __init__(self, **kwargs):
        # 初始化父类
        super().__init__(**kwargs)
        # 定义模型参数
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
    
    # 定义模型的前向计算
    def forward(self, x):
        x = self.hidden(x)
        x = self.act(x)
        o = self.output(x)
        return o  


In [44]:
import torch.onnx
# netron 可视化模型
X = torch.rand(2,784) # 设置一个随机的输入张量
net = MLP() # 实例化模型
print(net) # 打印模型
net(X) # 前向计算
torch.onnx.export(net, X, "model_mlp.onnx")

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


使用 Module 来自定义层

In [11]:
"""
不含模型参数的层
MyLayer 类通过继承 Module 类自定义一个将输入减掉均值后输出的层，
并将层的计算定义在了 forward 函数里。这个层里不含模型参数。
"""
class MyLayer(nn.Module):
    def __init__(self, **kwargs):
        # 初始化父类
        super().__init__(**kwargs)
    
    # 定义模型的前向计算
    def forward(self, x):
        return x - x.mean()
    
# 实例化模型
layer = MyLayer()
# 前向计算
x = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
y = layer(x)
print(y) # 打印输出

tensor([-2., -1.,  0.,  1.,  2.])


In [15]:
"""
自定义含模型参数的自定义层。其中的模型参数可以通过训练学出
Parameter 类其实是 Tensor 的子类，如果一个 Tensor 是 Parameter ，那么它会自动被添加到模型的参数列表里。
所以在定义含模型参数的层时，应该将参数定义成 Parameter。
除了直接定义成 Parameter 类外，还可以使用类 ParameterList 和 ParameterDict 分别定义参数的列表和字典。
"""
class MyListDense(nn.Module):
    def __init__(self, **kwargs):
        # 初始化父类
        super().__init__(**kwargs)
        # 定义模型参数
        #创建一个参数列表
        self.param_list = nn.ParameterList()
        #创建三个4x4参数，并加入到参数列表中
        for i in range(3):
            self.param_list.append(nn.Parameter(torch.randn(4, 4)))
        #创建一个4x1参数，并加入到参数列表中
        self.param_list.append(nn.Parameter(torch.randn(4, 1)))

    # 定义模型的前向计算
    def forward(self, x):
        # 计算参数列表的矩阵乘法
        for i in range(len(self.parameters)):
            x = torch.mm(x, self.parameters[i])
        return x
    
# 实例化模型
layer = MyListDense()
print(layer)


class MyDictDense(nn.Module):
    def __init__(self, **kwargs):
        # 初始化父类
        super().__init__(**kwargs)
        # 定义模型参数
        # 创建一个参数字典
        self.param_dict = nn.ParameterDict({"Linear1": nn.Parameter(torch.randn(4, 4)),
                                            "Linear2": nn.Parameter(torch.randn(4, 1))})
        self.param_dict["Linear3"] = nn.Parameter(torch.randn(4, 2))

    def forward(self, x, choice="Linear1"):
        return torch.mm(x, self.param_dict[choice])
    
# 实例化模型
layer = MyDictDense()
print(layer)

MyListDense(
  (param_list): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4]
      (1): Parameter containing: [torch.float32 of size 4x4]
      (2): Parameter containing: [torch.float32 of size 4x4]
      (3): Parameter containing: [torch.float32 of size 4x1]
  )
)
MyDictDense(
  (param_dict): ParameterDict(
      (Linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (Linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (Linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


常见神经网络层

卷积层

In [ ]:
"""
二维卷积层
二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。
卷积层的模型参数包括了卷积核和标量偏差。
在训练模型时，通常先对卷积核随机初始化，然后不断迭代卷积核和偏差
实际应用中常使用PyTorch内置的torch.nn.Conv2d，效率更高且支持更多功能
"""
# 例如输入矩阵 
# [[1,2],[3,4]]与卷积核 [[−1,0],[0,1]]运算
# Y = [[−1*1+0*2+0*3+1*4]] = [[3]]
# 输出尺寸
# H_out = (H_in + 2*padding - kernel_size_h) / stride + 1
# W_out = (W_in + 2*padding - kernel_size_w) / stride + 1
# ( p )：填充（Padding）数
# ( s )：步长（Stride）


# 计算二维卷积（互相关）
def coor2d(X, K):
    # 获取卷积核的形状
    k_h, k_w = K.shape
    X, K = X.float(), K.float()
    # 获取输出的形状
    Y = torch.zeros(X.shape[0] - k_h + 1, X.shape[1] - k_w + 1)
    # 进行卷积运算
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + k_h, j:j + k_w] * K).sum()
    return Y

# 二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size, **kwargs):
        # 初始化父类
        super().__init__(**kwargs)
        # 定义模型参数
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
    
    # 定义模型的前向计算
    def forward(self, x):
        return coor2d(x, self.weight) + self.bias



In [24]:
"""
创建一个⾼和宽为3的二维卷积层，
设输⼊高和宽两侧的填充数分别为1。
给定一个高和宽为8的输入，输出的高和宽也是8
"""

import torch
import torch.nn as nn

# 定义函数计算二维卷积层，对输入和输出做相应升维和降维
def com_conv2d(conv2d, X):
    # 升维
    # （1，1，h，w）表示一个批量大小为1，通道数为1的输入
    X = X.view((1, 1) + X.shape)
    # 计算卷积
    Y = conv2d(X)
    # 降维
    # 排除不关心的前两维:批量和通道
    Y = Y.view(Y.shape[2:])
    return Y

# 实例化卷积层
# 注意这里是两侧分别填充1⾏或列，所以在两侧一共填充2⾏或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 3), padding=1)
X = torch.randn(8, 8)
print(com_conv2d(conv2d, X).shape) # 输出卷积后的形状

# 当卷积核的高和宽不同时，我们也可以通过设置高和宽上不同的填充数使输出和输入具有相同的高和宽
# 使用高为5、宽为3的卷积核。在⾼和宽两侧的填充数分别为2和1
conv2d_1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5, 3), padding=(2, 1))
X = torch.randn(8, 8)
print(com_conv2d(conv2d_1, X).shape) # 输出卷积后的形状


torch.Size([8, 8])
torch.Size([8, 8])


池化层

In [39]:
"""
池化层每次对输入数据的一个固定形状窗口(⼜称池化窗口)中的元素计算输出。
不同于卷积层里计算输⼊和核的互相关性，池化层直接计算池化窗口内元素的属性（均值、最大值等）。
常见的池化包括最大池化或平均池化。
在二维最大池化中，池化窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输⼊数组上滑动。
当池化窗口滑动到某⼀位置时，窗口中的输入子数组的最大值即输出数组中相应位置的元素。
"""
import torch
import torch.nn as nn

# 定义函数计算二维池化层
def pool2d(X, pool_size, mode='max'):
    # 获取池化窗口的形状
    p_h, p_w = pool_size
    # 获取输出的形状
    Y = torch.zeros((X.shape[0] - p_h) + 1, (X.shape[1] - p_w) + 1)
    # 进行池化运算
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                # 计算池化窗口内的最大值
                # X[i: i + p_h, j: j + p_w]表示池化窗口内的元素
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

# 实例化池化层
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]], dtype=torch.float)
print(pool2d(X, (2, 2)))
print(pool2d(X, (2, 2), mode='avg')) # 平均池化

# 使用PyTorch内置的最大池化
# PyTorch池化层要求输入为 4D张量：格式：(batch_size, channels, height, width)
#input_4d = X.unsqueeze(0).unsqueeze(0)
input_4d = X.view(1, 1, 3, 3) # 升维
print(input_4d.shape)
max_pool = nn.MaxPool2d(kernel_size=(2, 2), stride=1) # 使用PyTorch内置的最大池化
max_output = max_pool(input_4d) # 计算池化
print(max_output.squeeze())

tensor([[4., 5.],
        [7., 8.]])
tensor([[2., 3.],
        [5., 6.]])
torch.Size([1, 1, 3, 3])
tensor([[4., 5.],
        [7., 8.]])


LeNet模型示例

原始LeNet-5结构（论文版本）：
Input (32×32 Grayscale)
→Conv6 (5×5)→C1: 28×28×6
→AvgPool→S2: 14×14×6
→Conv16 (5×5)→C3: 10×10×16
→AvgPool→S4: 5×5×16
→Conv120 (5×5)→C5: 1×1×120
→FC→F6: 84→Output→10 classes


In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class my_lenet(nn.Module):
    def __init__(self):
        super().__init__()
        # nn.Sequential 是PyTorch中用于构建顺序执行模型结构的容器类
        self.feature = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5), # 卷积层
            nn.Tanh(), # 激活函数
            nn.AvgPool2d(kernel_size=2, stride=2), # 平均池化层
            nn.Conv2d(6, 16, kernel_size=5), # 卷积层
            nn.Tanh(), # 激活函数
            nn.AvgPool2d(kernel_size=2, stride=2), # 平均池化层
        )

        self.classifier = nn.Sequential(
            nn.Flatten(), # 展平层
            nn.Linear(16 * 5 * 5, 120), # 全连接层
            nn.Tanh(), # 激活函数
            nn.Linear(120, 84), # 全连接层
            nn.Tanh(), # 激活函数
            nn.Linear(84, 10) # 全连接层
        )

    def forward(self, x):
        # 前向传播
        x = self.feature(x)
        x = self.classifier(x)
        return x
    

net_lenet = my_lenet()
print(net_lenet)
    
# 模型的可学习参数可以通过net.parameters()返回
params = list(net_lenet.parameters())
print(params[0])
print(len(params))
print(params[0].size())# conv1的权重
    

my_lenet(
  (feature): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Tanh()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Tanh()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)
Parameter containing:
tensor([[[[ 0.0145,  0.1264, -0.1493,  0.0898,  0.0716],
          [ 0.0741, -0.0731, -0.0322,  0.0336,  0.1859],
          [-0.0681, -0.0216, -0.0402,  0.1715,  0.1574],
          [-0.0004, -0.0124,  0.1425, -0.0507,  0.1439],
          [-0.0354,  0.1988,  0.1777, -0.0116,  0.1631]]],


        [[[ 0.1773,  0.1258,  0.1941,  0.1336,  0.0020],
          [ 0.0519, -0.0488, -0.0347, -0.0185

In [51]:
input = torch.randn(1, 1, 32, 32)
# 前向传播
out = net_lenet(input)
torch.onnx.export(net_lenet, input, "model_lenet.onnx")

#清零所有参数的梯度缓存，然后进行随机梯度的反向传播
net_lenet.zero_grad()
out.backward(torch.randn(1, 10))

AlexNet

In [52]:
"""网络结构
输入层：227×227×3的RGB图像
层间结构（按顺序排列）：

卷积层1
卷积核：11×11，步长4，96个通道
输出尺寸：55×55×96
激活函数：ReLU
池化层：3×3最大池化，步长2 → 输出27×27×96
局部响应归一化（LRN）

卷积层2
卷积核：5×5，填充2，256个通道
输出尺寸：27×27×256
激活函数：ReLU
池化层：3×3最大池化，步长2 → 输出13×13×256
局部响应归一化（LRN）

卷积层3
卷积核：3×3，填充1，384个通道
输出尺寸：13×13×384
激活函数：ReLU

卷积层4
卷积核：3×3，填充1，384个通道
输出尺寸：13×13×384
激活函数：ReLU

卷积层5
卷积核：3×3，填充1，256个通道
输出尺寸：13×13×256
激活函数：ReLU
池化层：3×3最大池化，步长2 → 输出6×6×256

全连接层
FC6：4096个神经元（输入维度：6×6×256=9216）
Dropout率：0.5
FC7：4096个神经元
Dropout率：0.5
FC8：1000个神经元（对应ImageNet类别）
"""
class my_alexnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4), # 卷积层1
            nn.ReLU(), # 激活函数
            nn.MaxPool2d(kernel_size=3, stride=2), # 最大池化层
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2), # 局部响应归一化
            nn.Conv2d(96, 256, kernel_size=5, padding=2), # 卷积层2
            nn.ReLU(), # 激活函数
            nn.MaxPool2d(kernel_size=3, stride=2), # 最大池化层
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2), # 局部响应归一化
            nn.Conv2d(256, 384, kernel_size=3, padding=1), # 卷积层3
            nn.ReLU(), # 激活函数
            nn.Conv2d(384, 384, kernel_size=3, padding=1), # 卷积层
            nn.ReLU(), # 激活函数
            nn.Conv2d(384, 256, kernel_size=3, padding=1), # 卷积层5
            nn.ReLU(), # 激活函数
            nn.MaxPool2d(kernel_size=3, stride=2), # 最大池化层
        )
        self.classifier = nn.Sequential(
            nn.Flatten(), # 展平层
            nn.Linear(256 * 6 * 6, 4096), # 全连接层
            #相比于LeNet，全连接层的输出个数大数倍。使用丢弃层来缓解过拟合
            nn.Dropout(0.5), # Dropout层
            nn.ReLU(), # 激活函数
            nn.Linear(4096, 4096), # 全连接层
            nn.Dropout(0.5), # Dropout层
            nn.ReLU(), # 激活函数
            nn.Linear(4096, 1000) # 全连接层
        )

    def forward(self, x):
        x = self.feature(x)
        x = self.classifier(x)
        return x
    
net_alexnet = my_alexnet()
print(net_alexnet)
input = torch.randn(1, 3, 227, 227)
# 前向传播
out = net_alexnet(input)
torch.onnx.export(net_alexnet, input, "model_alexnet.onnx")
# 清零所有参数的梯度缓存，然后进行随机梯度的反向传播
net_alexnet.zero_grad()
out.backward(torch.randn(1, 1000))

my_alexnet(
  (feature): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

/home/zcx/Apps/anaconda/envs/universe/lib/python3.8/site-packages/torch/onnx/_internal/jit_utils.py:314: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789259345/work/torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/zcx/Apps/anaconda/envs/universe/lib/python3.8/site-packages/torch/onnx/utils.py:739: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789259345/work/torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/home/zcx/Apps/anaconda/envs/universe/lib/python3.8/site-packages/torch/onnx/utils.py:1244: UserWarning: Constant folding - Only steps=1 can be constant folded for opse

模型初始化

In [60]:
"""
torch.nn.init提供了以下初始化方法： 
1. torch.nn.init.uniform_(tensor, a=0.0, b=1.0) 
2. torch.nn.init.normal_(tensor, mean=0.0, std=1.0) 
3. torch.nn.init.constant_(tensor, val) 
4. torch.nn.init.ones_(tensor) 
5. torch.nn.init.zeros_(tensor) 
6. torch.nn.init.eye_(tensor) 
7. torch.nn.init.dirac_(tensor, groups=1) 
8. torch.nn.init.xavier_uniform_(tensor, gain=1.0) 
9. torch.nn.init.xavier_normal_(tensor, gain=1.0) 
10. torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan__in', nonlinearity='leaky_relu') 
11. torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu') 
12. torch.nn.init.orthogonal_(tensor, gain=1) 
13. torch.nn.init.sparse_(tensor, sparsity, std=0.01)
14. torch.nn.init.calculate_gain(nonlinearity, param=None)
"""

"\ntorch.nn.init提供了以下初始化方法： \n1. torch.nn.init.uniform_(tensor, a=0.0, b=1.0) \n2. torch.nn.init.normal_(tensor, mean=0.0, std=1.0) \n3. torch.nn.init.constant_(tensor, val) \n4. torch.nn.init.ones_(tensor) \n5. torch.nn.init.zeros_(tensor) \n6. torch.nn.init.eye_(tensor) \n7. torch.nn.init.dirac_(tensor, groups=1) \n8. torch.nn.init.xavier_uniform_(tensor, gain=1.0) \n9. torch.nn.init.xavier_normal_(tensor, gain=1.0) \n10. torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan__in', nonlinearity='leaky_relu') \n11. torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu') \n12. torch.nn.init.orthogonal_(tensor, gain=1) \n13. torch.nn.init.sparse_(tensor, sparsity, std=0.01)\n14. torch.nn.init.calculate_gain(nonlinearity, param=None)\n"

In [ ]:
"""
torch.nn.init中为我们提供了常用的初始化方法
根据实际模型来使用torch.nn.init进行初始化，
通常使用isinstance()来进行判断模块（回顾3.4模型构建）属于什么类型
对于不同的类型层，可以设置不同的权值初始化的方法。
"""
import torch
import torch.nn as nn

conv = nn.Conv2d(1,3,3)
linear = nn.Linear(10,1)

print(isinstance(conv,nn.Conv2d)) # 判断conv是否是nn.Conv2d类型
print(isinstance(linear,nn.Conv2d)) # 判断linear是否是nn.Conv2d类型

# Conv默认使用Kaiming均匀初始化（针对ReLU激活函数）
# torch.nn.init.kaiming_uniform__(conv.weight.data, a=0, mode='fan_in', nonlinearity='relu')
# Linear层默认使用PyTorch的默认初始化（类似均匀分布[-sqrt(1/in_features), sqrt(1/in_features)]）
# torch.nn.init.uniform_(tensor, a=0.0, b=1.0) 

# 查看默认初始化的conv参数
print(conv.weight.data)
# 查看linear的参数
print(linear.weight.data)

# 使用torch.nn.init对conv进行kaiming初始化
# Conv2d使用Kaiming正态分布初始化（均值为0，标准差由非线性函数类型决定）
# Linear层使用固定常数0.3初始化
torch.nn.init.kaiming_normal_(conv.weight.data)
print(conv.weight.data)
# 对linear进行常数初始化
torch.nn.init.constant_(linear.weight.data,0.3)
print(linear.weight.data)


True
False
tensor([[[[ 0.1198,  0.1576,  0.3134],
          [ 0.1904,  0.2399, -0.1122],
          [-0.2739, -0.1738, -0.1941]]],


        [[[-0.2303, -0.2003, -0.2437],
          [ 0.2278, -0.0374, -0.2509],
          [ 0.0449, -0.1777, -0.0911]]],


        [[[-0.2894, -0.0926, -0.0481],
          [-0.1950,  0.2164,  0.2436],
          [ 0.3228, -0.1256,  0.2833]]]])
tensor([[-0.2471,  0.0694, -0.1394, -0.2498, -0.2134, -0.2553, -0.3059, -0.1799,
         -0.2508, -0.3075]])
tensor([[[[-0.2425, -0.4311, -0.2917],
          [ 0.3703,  0.3474,  0.2775],
          [-0.0108,  0.3146, -0.3175]]],


        [[[-0.2880,  0.5921, -0.0472],
          [-0.7609,  1.0768,  0.7984],
          [ 1.0630,  0.2771, -0.5665]]],


        [[[-0.2564, -0.3100,  0.5399],
          [ 0.2551,  0.2877, -0.5882],
          [-0.3625, -0.3315,  0.1882]]]])
tensor([[0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
         0.3000]])


初始化函数封装

In [ ]:
"""
将各种初始化方法定义为一个initialize_weights()的函数并在模型初始后进行使用
遍历当前模型的每一层，然后判断各层属于什么类型，然后根据不同类型层，设定不同的权值初始化方法
在初始化时，最好不要将模型的参数初始化为0，因为这样会导致梯度消失，从而影响模型的训练效果。
可以使用其他初始化方法或者将模型初始化为一个很小的值，如0.01，0.1等
"""
# 模型的定义
class MLP(nn.Module):
  # 声明带有模型参数的层，这里声明了两个全连接层
  def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Conv2d(1,1,3)
    self.act = nn.ReLU()
    self.output = nn.Linear(10,1)
    
   # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
  def forward(self, x):
    o = self.act(self.hidden(x))
    return self.output(o)

mlp = MLP()
print(mlp.hidden.weight.data)
print("-------初始化-------")

def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_uniform_(m.weight.data, a=0, mode='fan_in', nonlinearity='relu')
        elif isinstance(m, nn.Linear):
            nn.init.uniform_(m.weight.data, a=-0.1, b=0.1)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.uniform_(m.weight.data, a=0.1, b=0.1)
            nn.init.constant_(m.bias.data, 0)

# 初始化自定义模型参数
initialize_weights(mlp)
print(mlp.hidden.weight.data)


tensor([[[[ 0.0696,  0.1003,  0.0408],
          [ 0.0235, -0.0515, -0.2017],
          [ 0.0274,  0.1809,  0.2459]]]])
-------初始化-------
tensor([[[[-0.5748, -0.3484,  0.7871],
          [ 0.3020,  0.4273,  0.0608],
          [ 0.1157,  0.2459,  0.3186]]]])
